# PyTorch 初心者ハンズオン

## はじめての画像分類
---

## ページ内目次

<ul>
<li><a href="#画像ファイルの読み込み">画像ファイルの読み込み</a></li>
<li><a href="#PyTorchのインポート、VGG16モデルのロード">PyTorchのインポート、VGG16モデルのロード</a></li>
<li><a href="#モデルを使って画像のクラスを予測する">モデルを使って画像のクラスを予測する</a></li>
<li><a href="#Tensorクラス中の数値データへのアクセス">Tensorクラス中の数値データへのアクセス</a></li>
<li><a href="#複数画像をバッチ処理する">複数画像をバッチ処理する</a></li>
<li><a href="#modelが返す行列を理解する">modelが返す行列を理解する</a></li>
<li><a href="#modelが返した確率をグラフ表示する">modelが返した確率をグラフ表示する</a></li>
<li><a href="#ラベルリストの利用">ラベルリストの利用</a></li>
<li><a href="#Top-N クラスの調査">Top-N クラスの調査</a></li>
<li><a href="#matplotlib-による円グラフの表示">matplotlibによる円グラフの表示</a></li>
<li><a href="#さらなる演習">さらなる演習</a></li>
</ul>

<hr>

PyTorch 初心者ハンズオンにようこそ！

本ハンズオンでは、ディープラーニングやPyTorch フレームワークを利用したことがない方を対象として、ディープラーニングによる画像分類の方法をご紹介します。

<hr>

画像に何が映っているか、深層学習によって推測させてみましょう。PyTorchを使った場合、有名なネットワーク構造がいくつか、すぐに試せる状態になっています。ここでは、2015年に発表された VGG16 を使ってみましょう。

<div style="border: 1px solid; padding: 10px">
VGG16ネットワークは、224x224の画像を1000クラスに分類するためのネットワークです。
<p>
Very Deep Convolutional Networks for Large-Scale Image Recognition<br>https://arxiv.org/abs/1409.1556

<p>PyTorch では torchvision.models.vgg16 クラスが定義されています。 PyTorch 自体に学習済みデータ組み込まれていませんが、公開されている学習済みモデルを読み込んで利用できるため、学習に時間をかけなくても試せます。

https://github.com/pytorch/vision/blob/master/torchvision/models/vgg.py

</div>

<hr>

### 画像ファイルの読み込み

In [ ]:
from google.colab import drive
drive_path = '/content/drive/'
drive.mount(drive_path)
pj_path = drive_path + 'My Drive/ColabNotebooks/PyTorchBeginner/'

In [ ]:
from PIL import Image

img = Image.open(pj_path + "_images/cat.jpg")

In [ ]:
img

上記ブロックを実行し、猫が表示されましたか？

一応、サイズを確認しておきましょう。

In [ ]:
img.size

---

### PyTorchのインポート、VGG16モデルのロード


PyTorchのインポート、訓練済みモデルのロード

In [ ]:
import torch as t
import torchvision as tv

ここで、 model にはVGG16 と呼ばれる有名な画像分類ネットワークがロードします。

In [ ]:
model = tv.models.vgg16(pretrained=True)

モデルの確認

model オブジェクトが torchvision.models.vgg.VGG クラスであることを確認してみてください。

In [ ]:
type(model)

機械学習では、モデルを使用するタイミングは二つあり、PyTorchではそれぞれを切り替える必要があります。


- 学習フェイズ  
  整備されたデータを使って正しい結論を導けるようにパラメータを調整するフェイズ
- 推論フェイズ  
  調整されたパラメータを使って新しいデータを判定するフェイズ

今回は学習済みのモデルを使用するので推論フェイズになります。

> Some models use modules which have different training and evaluation behavior, such as batch normalization. To switch between these modes, use model.train() or model.eval() as appropriate. See train() or eval() for details.
> https://pytorch.org/docs/master/torchvision/models.html

In [ ]:
model.eval()

パワーポイントのモデルと見比べてみましょう。
画像のサイズは書かれていませんが、`Conv2d`の第１引数と第２引数から3次元目の変化が見て取れます。

featuresのあとはclassifierが続けて実行されます。
結果、1000クラスまでの分類ができるNNとなっています。

一点だけ注意しなければならないのは、資料では出力レイヤーはsoftmaxと書かれていますが、PyTorchのネットワーク構造はそうはなっていないということです。  
プログラマーが自力でsoftmaxを適用する必要があります。

推論自体はそんなに重たい処理ではないのでCPUでも十分な速度が出ます。  
しかし、次の「学習」を学ぶ過程ではCPUでは荷が重いので、ここでCPUとGPUの切り替えを学んでおきましょう。

In [ ]:
device = t.device("cuda:0" if t.cuda.is_available() else "cpu")
device

In [ ]:
model = model.to(device)

---

### 前処理を行う


PyTorchでは、画像をテンソル型にする必要がある。  
前処理をまとめて定義しておく。

> https://pytorch.org/docs/master/torchvision/models.html

In [ ]:
preprocess = tv.transforms.Compose([
                                  tv.transforms.Resize(256),
                                  tv.transforms.CenterCrop(224),
                                  tv.transforms.ToTensor(),
                                  tv.transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
])

前処理を実行して、画像からテンソルを取得してみる。  
前処理自体はCPUに行わせます。

In [ ]:
img_tensor = preprocess(img)
img_tensor.shape

tensorの順番は少し直観的ではないかもしれない。  
CH, Height, Widthの順番に格納されているので、さっき確認した画像サイズと照らし合わせてみよう。

画像をモデルに入力するときは、3次元テンソルではなく、４次元テンソルである必要がある。  
何の要素が必要かというと、入力する画像の枚数データである。  
これを**バッチサイズ**と呼ぶため、必ず覚えておいてほしい。

今回は１枚だけを入力するので、0次元目に１を新規要素として追加する。


In [ ]:
img_tensor.unsqueeze_(0)
img_tensor.size()

---

### モデルを使って画像のクラスを予測する

テンソルをニューラルネットワークに入力し、出力 p を得る

In [ ]:
p = model(img_tensor.to(device))

※ modelのCPU実行には、いくらか時間がかかります。

予測結果（の生データ）を表示してみる

In [ ]:
p.shape

In [ ]:
p

たくさんの値がはいった行列（Tensor型）が見えるはずです。

---

### Tensorクラス中の数値データへのアクセス

Tensor クラスはPyTorchが行列を扱うための構造です。

In [ ]:
p.__class__

In [ ]:
p.detach().__class__

データの他にも、勾配を算出する関数も持っていることが分かる。

In [ ]:
p[0,1]

In [ ]:
p.detach()[0,1]

---

### 複数画像をバッチ処理する

複数の画像を連続で処理したい場合は、配列として画像を複数渡して、まとめて予測を得ることができます。この際、 modelが返すテンソル（イメージ数、 1000クラス）になります。

1枚単位で画像を予測する場合と結果は一緒ですが、GPU搭載サーバーで短時間に多数の画像を予測したい場合、多数の画像をまとめることにより、システム全体のスループットが向上します。

#### 1枚のみ処理する場合

In [ ]:
img_tensor1 = preprocess(img)
img_tensor1.unsqueeze_(0)
img_tensor.shape

生成したモデルをNvidiaのGPUに転送しましょう。

In [ ]:
p1 =  model(img_tensor1.to(device))

In [ ]:
p1.shape # => (1, 000)

#### 2枚処理する場合

操作は基本的にNumpyに準拠しているため、それと同じように次元を操作する。  
今回は、新しい次元を軸にテンソルを連結したいため、stack関数を用いる。  
まず、前に解説したSeriesの次元が2になっていることを確認しよう。

In [ ]:
img_tensor2 = preprocess(img)
img_tensor2 = t.stack([img_tensor2, img_tensor2])
img_tensor2.shape

In [ ]:
p2 =  model(img_tensor2.to(device))

In [ ]:
p2.shape # 演習: 結果を予想してください

#### 3枚処理する場合

【演習】  
３枚の画像をmodelに渡して、期待通りのテンソルが戻されることを確認してください。

In [ ]:
img_tensor3 = preprocess(img)
img_tensor3 = t.stack([img_tensor3, img_tensor3, img_tensor3])
p3 = model(img_tensor3.to(device))
p3.shape

---

### modelが返す行列を理解する


以降、予測した最初の画像 (0行目) の内容について調査していきます。  
変数 d に p.detach()[0] を代入し、以降の操作は d に対して行います。


In [ ]:
d = p.detach()[0]

In [ ]:
d.shape # dの要素数を確認したい方はお試しください

In [ ]:
d # 中身を見たい方はお試しください

PyTorchに足りなかったsoftmax処理を実行しましょう。  
softmax処理によってデータが確率値になります。

> 厳密には確率ではありません。

In [ ]:
import torch.nn.functional as F

In [ ]:
softmax_d = F.softmax(d, dim=0)
softmax_d[:5]

ここまでで出力層までの計算処理が終わりました。

d を棒グラフとして表示してみます。

### modelが返した確率をグラフ表示する

棒グラフの表示は`matplotlib`で行います。  

これから出力されたデータを見ていきますが、`matplotlib`だけでなく、通常のライブラリはCUDAに対応していません。  
NNの重たい処理も終わっているので、ここでGPUからCPUにデータを移してしまいます。

In [ ]:
softmax_d = softmax_d#.cpu() # 今までと違ってベタ指定
softmax_d[:5]

`device='cuda:0'`がなくなったことを確認してください。

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

plt.style.use('ggplot')
plt.figure(figsize=(20,10))
plt.bar(x=range(len(softmax_d)), height=softmax_d)
plt.plot()

<div style="border: 1px solid; padding: 10px">これは、0番目から999番目のクラスまで、そのクラスである確率の棒グラフです。200 ~ 400 の間にもっとも高いバーが見えているはずですが、この最も高い値がいくつめの要素か調べていきます。</div>

配列 d の中から最大値を探す


In [ ]:
import numpy as np

In [ ]:
predicted_label = np.argmax(softmax_d) # Tensor型 -> Numpy型へ自動変換

最大値がある要素の番号を表示


In [ ]:
predicted_label

d[predicted_label] は d の中の最大値で、確率（0.0〜1.0）


In [ ]:
softmax_d[predicted_label]

`np.argmax(softmax_d)` が 281  を返した場合、入力した画像 img は 281 番目のクラスに分類されたことを意味しています。

---

## ラベルリストの利用

入力した画像は、 predicted_label 番目（281番目）のクラスである確率が高いことがわかりました。  
しかし、この 281 番目のクラスというのは何を指しているのでしょうか。

（このモデルの学習時に利用された）クラスラベルリスト <a href="synset_words.txt">synset_words.txt</a>  と照合することで、このクラスラベルが何を意味しているのかを調べます。

In [ ]:
with open(pj_path+"synset_words.txt", 'r') as f:
    labels = f.readlines()

labels の内容を確認する (1000個まで表示すると長いので、とりあえず最初の10個まで)

In [ ]:
labels[:10]

入力画像に対して最も高い確率を示したクラスのラベル文字列を表示してみよう。  
predict_label 番目の要素を調べる


In [ ]:
labels[predicted_label]

実行した結果、tabby, tabby cat、つまり、ニューラルネットワークは、この画像が  tabby, tabby catに分類した、ということになります。

<hr>

## Top-N クラスの調査

<div style="border: 1px solid; padding: 10px">返された値は「入力された画像が各クラスの値である確率」だと説明しました。
また、最大値は 281番目の要素であることがわかりました。最大値に続く、大きな値を探してみましょう。</div>

確率を昇順にソートして、最後の最大3件を表示


In [ ]:
softmax_d.topk(3)

<div style="border: 1px solid; padding: 10px">このままではクラスラベルがなく、確率値が何に対応しているのかがわかりません。以下の操作で確率とクラスラベルをマージした上でソートしてみましょう。</div>

確率とクラスラベルを組みわせたリストの作成



In [ ]:
p2 = list(zip(softmax_d, labels))
p2[:10]

作成した組み合わせリストから Top 3 を表示

`topk`を使わなくても、リスト形式の演算もできます。

In [ ]:
sorted(p2, reverse=True)[:3]

<div style="border: 1px solid; padding: 10px">
出力例

<p>
[<br>(0.52825135, 'n02124075 Egyptian cat\n'), <br>
   (0.11501167, 'n02123045 tabby, tabby cat\n'), <br>
   (0.052949127, 'n02123159 tiger cat\n')<br>]
</div>

<hr>

【演習】<br><br>同様に、Top-10 クラスを確認してください。

In [ ]:
p2 = list(zip(softmax_d, labels))
sorted(p2, reverse=True)[:10]

<hr>

## matplotlib による円グラフの表示

<div style="border: 1px solid; padding: 10px">降順にソートした確率一覧を matplotlib の円グラフにすることで、Top-Nクラスがどれぐらいの割合を占めるのかを視覚的に確認できます。</div>

In [ ]:
sorted_softmax_d = sorted(softmax_d, key=lambda x:-x)
sorted_softmax_d[:10]

In [ ]:
plt.style.use('ggplot')
plt.rcParams.update({'font.size':15})
plt.figure(figsize=(10,10))
plt.pie(sorted_softmax_d)
plt.plot()

<hr>

## さらなる演習

【演習】  
ハンズオン環境に用意されている画像を model にわたし、各画像の予測クラス（１画像あたり1クラス）を表示してください。


ハンズオン環境に存在する画像</a>をまとめて読み込みたい場合は、次のブロックを実行してください。

In [ ]:
img_airplane = Image.open(pj_path+"_images/airplane.jpg")
img_cat = Image.open(pj_path+"_images/cat.jpg")
img_dog = Image.open(pj_path+"_images/dog.jpg")
img_dolphin = Image.open(pj_path+"_images/dolphin.jpg")
img_human1 = Image.open(pj_path+"_images/human_1.jpg")
img_human2 = Image.open(pj_path+"_images/human_2.jpg")
img_spider = Image.open(pj_path+"_images/spider.jpg")

In [ ]:
# 演習
img_tensor_ = preprocess(img_airplane).cuda()
img_tensor_.unsqueeze_(0)
p_ = model(img_tensor_)